# Programming and Database Fundamentals for Data Scientists - EAS503

## Installing iPython (jupyter) notebook:

Start by installing `Python 3`. Easiest way is to install `Anaconda Python`, which is a free and open source distribution of Python, consisting of many useful libraries.

Follow the instructions at: [https://docs.continuum.io/anaconda/install/](https://docs.continuum.io/anaconda/install/)

You can start IPython notebook by running

```shell
jupyter notebook
```


## Setting up sqlite database
In this course we will use `sqlite` database. You can install the package `sqlite3` it on your system, following instructions here:
[https://sqlite.org/index.html](https://sqlite.org/index.html) or if you need a GUI, go for [https://razorsql.com/features/sqlite_features.html](https://razorsql.com/features/sqlite_features.html).

After this step, you should have a sqlite database running on your laptop.
### Installing python bindings for mysql
To be able to connect to the mysql database, you need to install the `sqlite3` python package that will let you connect to the database from within a python application. More information here:

[https://docs.python.org/2/library/sqlite3.html](https://docs.python.org/2/library/sqlite3.html)

In Unix-like environments (including MacOS), you can try:
```shell
pip install sqlite
```
or
```shell
conda install sqlite
```
Might need `sudo` privileges depending on your Python installation.


## Demonstrating a simple data science pipeline.
Data available from [Chicago Crime Data](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2)

Before starting to play with the Chicago data, you will need to import the data into your database. We will assume that the database server is running on localhost on the standard port (see mysql help for handling other settings), the username is `username` and the password is `password` (You should use better usernames and passwords!)

### Step 1: Getting the data
Download the data from [here](https://www.cse.buffalo.edu/ubds/docs/chicago_crime_data.csv). This is only data for 2015 onwards. You can also download the full dataset here as well [here](https://www.cse.buffalo.edu/ubds/docs/chicago_crime_data_all.csv).

### Step 2: Setting up the database
Create a new database and the empty table using the lines below. 

In [50]:
conn = sqlite3.connect('/Users/chandola/Teaching/eascourse/scripts/eas503db.db')
c = conn.cursor()

c.execute('''create table `chicago_crime_data` (
  ID mediumint(9),
  Case_Number text,
  Date varchar(32),
  Block text,
  IUCR text,
  Primary_Type text,
  Description text,
  Location_Description text,
  Arrest varchar(6),
  Domestic varchar(6),
  Beat text,
  District text,
  Ward int,
  Community_Area text,
  FBI_Code text,
  X_Coordinate float,
  Y_Coordinate float,
  Year int(4),
  Updated_On varchar(32),
  Latitude float,
  Longitude float,
  Location varchar(64)
);''')
conn.commit()

### Step 3: Importing data into the database.

In [51]:
import pandas as pd
import numpy as np
df = pd.read_csv('chicago_crime_data.csv',header=0)

In [52]:
data = np.array(df)

In [53]:
c.executemany('INSERT INTO `chicago_crime_data` (ID,Case_Number,Date,Block,IUCR,Primary_Type,Description,Location_Description,Arrest,Domestic,Beat,District,Ward,Community_Area,FBI_Code,X_Coordinate,Y_Coordinate,Year,Updated_On,Latitude,Longitude,Location) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);', data)

In [54]:
conn.commit()

### Let us start with a very simple query
Get the total number of rows in the database

In [55]:
res = c.execute('SELECT COUNT(*) FROM `chicago_crime_data`')

In [56]:
print(res.fetchall())

[(436264,)]


### Doing more interesting things with SQL
Which are the more crime ridden areas in Chicago since 2016?

In [67]:
querystr = '''SELECT count(*) as cnt, District 
              FROM chicago_crime_data 
              WHERE strftime(Date) > strftime(\'01/01/2016 00:00:00 AM\') 
              GROUP BY District 
              ORDER BY cnt'''
c = conn.cursor()
res = c.execute(querystr)

In [68]:
for row in c.fetchall():
    print(row)

(5, '31')
(7108, '20')
(12350, '24')
(12546, '17')
(13846, '22')
(14826, '16')
(16799, '14')
(17883, '15')
(18493, '2')
(19176, '5')
(19678, '19')
(19999, '9')
(20195, '3')
(20344, '10')
(22188, '18')
(22449, '12')
(23003, '7')
(23175, '1')
(23496, '25')
(24144, '4')
(26632, '6')
(27914, '8')
(30015, '11')


### Zooming into Jefferson Park (District 11)

<img width="250" src='https://upload.wikimedia.org/wikipedia/commons/f/f8/US-IL-Chicago-CA11.svg'>

In [75]:
querystr = '''SELECT count(*) as cnt, Primary_Type FROM chicago_crime_data 
              WHERE District = "11" 
              AND strftime(Date) > strftime(\'01/01/2016 00:00:00 AM\')  
              GROUP BY Primary_Type 
              ORDER BY cnt'''
c = conn.cursor()
res = c.execute(querystr)

In [76]:
for row in c.fetchall():
    print(row)

(1, 'NON - CRIMINAL')
(1, 'NON-CRIMINAL')
(1, 'NON-CRIMINAL (SUBJECT SPECIFIED)')
(2, 'HUMAN TRAFFICKING')
(2, 'PUBLIC INDECENCY')
(3, 'CONCEALED CARRY LICENSE VIOLATION')
(4, 'OBSCENITY')
(11, 'INTIMIDATION')
(12, 'STALKING')
(18, 'LIQUOR LAW VIOLATION')
(21, 'KIDNAPPING')
(49, 'ARSON')
(74, 'SEX OFFENSE')
(86, 'GAMBLING')
(144, 'HOMICIDE')
(158, 'CRIM SEXUAL ASSAULT')
(203, 'OFFENSE INVOLVING CHILDREN')
(231, 'INTERFERENCE WITH PUBLIC OFFICER')
(274, 'PUBLIC PEACE VIOLATION')
(644, 'CRIMINAL TRESPASS')
(720, 'WEAPONS VIOLATION')
(745, 'PROSTITUTION')
(788, 'BURGLARY')
(849, 'DECEPTIVE PRACTICE')
(1212, 'MOTOR VEHICLE THEFT')
(1571, 'ROBBERY')
(1768, 'OTHER OFFENSE')
(2200, 'ASSAULT')
(2825, 'CRIMINAL DAMAGE')
(3327, 'THEFT')
(5707, 'NARCOTICS')
(6364, 'BATTERY')


In [80]:
conn.close()

### PYTHON Library PANDAS
One can directly read data from the csv file into a Pandas object

In [77]:
import pandas as pd

In [78]:
df = pd.read_csv('chicago_crime_data.csv',header=0)

In [79]:
df.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10496347,HZ237390,04/23/2016 04:55:00 PM,001XX N PARKSIDE AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,False,...,29.0,25,14,1138578.0,1900413.0,2016,04/30/2016 03:51:13 PM,41.882858,-87.766599,"(41.88285803, -87.766599362)"
1,10496348,HZ237355,04/23/2016 02:00:00 PM,0000X E RIVERWALK S,0820,THEFT,$500 AND UNDER,SIDEWALK,False,False,...,42.0,32,06,1176778.0,1902518.0,2016,04/30/2016 03:51:13 PM,41.887856,-87.626264,"(41.887856357, -87.626264274)"
2,10496349,HZ237341,04/23/2016 04:10:00 PM,003XX W 35TH ST,0495,BATTERY,AGGRAVATED OF A SENIOR CITIZEN,SPORTS ARENA/STADIUM,True,False,...,11.0,34,04B,1174431.0,1881739.0,2016,04/30/2016 03:51:13 PM,41.830890,-87.635503,"(41.830890037, -87.635503335)"
3,10496350,HZ237330,04/23/2016 01:30:00 PM,040XX N MAJOR AVE,031A,ROBBERY,ARMED: HANDGUN,ALLEY,False,False,...,38.0,15,03,1137626.0,1926291.0,2016,04/30/2016 03:51:13 PM,41.953887,-87.769470,"(41.953887423, -87.76947041)"
4,10496351,HZ237402,04/23/2016 04:45:00 PM,084XX S DREXEL AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,True,...,8.0,44,26,1183692.0,1849271.0,2016,04/30/2016 03:51:13 PM,41.741584,-87.602537,"(41.741583562, -87.602537135)"


Many of the operations that can be done as SQL queries on the database, can also be done on a `Pandas` object. Then what is benefit of a database?

## Measuring performance of a Python code
Here we will focus on two types of performance metrics:
#### Speed - time taken to run the code on the machine
We will use a Python tool called `timeit` to measure the time. 

We will first define two `functions` (snippets of code that can be called repeatedly) to use `mysql` and `Pandas`

In [82]:
def pd_performance():
    df = pd.read_csv('/Users/chandola/Teaching/eascourse/data/chicago_crime_data.csv',header=0)
    cnt = len(df.index)
    
def db_performance():
    conn = sqlite3.connect('/Users/chandola/Teaching/eascourse/scripts/eas503db.db')
    c = conn.cursor()
    querystr = 'SELECT count(*) FROM chicago_crime_data'
    c.execute(querystr)

    for row in c.fetchall():
        cnt = row
    conn.close()

In [83]:
timeit pd_performance()

2.11 s ± 60.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
timeit db_performance()

31.6 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Space - memory used to run the code on the machine
We will use another Python tool called `memory_profiler` for this. For more see - https://pypi.org/project/memory_profiler/. Conda users may install the tool using:
```shell
conda install memory_profiler
```

Unfortunately, the `memory_profiler` does not support notebooks. So we will run these two codes from the command line.

In [43]:
import pandas as pd
from memory_profiler import profile

@profile
def pd_performance():
    df = pd.read_csv('chicago_crime_data.csv',header=0)
    cnt = len(df.index)

pd_performance()

Save the above snippet into a file called `pd_demo.py`. Then run:
```shell
python -m memory_profiler pd_demo.py
```

In [ ]:
import sqlite3
from memory_profiler import profile

@profile
def db_performance():
    conn = sqlite3.connect('/Users/chandola/Teaching/eascourse/scripts/eas503db.db')
    c = conn.cursor()
    querystr = 'SELECT count(*) FROM chicago_crime_data'
    c.execute(querystr)

    for row in c.fetchall():
        cnt = row
    conn.close()

Save the above snippet into a file called `db_demo.py`. Then run:
```shell
python -m memory_profiler db_demo.py
```

### Observations:
The output of the `memory_profiler` gives us:

For `pd_demo.py`:
```
Line #    Mem usage    Increment   Line Contents
================================================
     4     77.6 MiB     77.6 MiB   @profile
     5                             def pd_performance():
     6    270.0 MiB    192.4 MiB       df = pd.read_csv('chicago_crime_data.csv',header=0)
     7    270.0 MiB      0.0 MiB       cnt = len(df.index)
```

For `db_demo.py`:
```
Line #    Mem usage    Increment   Line Contents
================================================
     4     49.5 MiB     49.5 MiB   @profile
     5                             def db_performance():
     6     49.5 MiB      0.0 MiB       db = pymysql.connect(host="127.0.0.1",    # your host, usually localhost
     7     49.5 MiB      0.0 MiB                        user="root",         # your username
     8     49.5 MiB      0.0 MiB                        passwd="root",       # your password
     9     49.7 MiB      0.2 MiB                        db="eas503db")    # name of the data base
    10     49.7 MiB      0.0 MiB       querystr = 'SELECT count(*) FROM chicago_crime_data'
    11     49.7 MiB      0.0 MiB       cur = db.cursor()
    12     49.7 MiB      0.0 MiB       cur.execute(querystr)
    13                             
    14     49.7 MiB      0.0 MiB       for row in cur.fetchall():
    15     49.7 MiB      0.0 MiB           cnt = row
    16     49.7 MiB      0.0 MiB       db.close()
```

Pulling data from the database is better, both in terms of memory usage and speed, than reading a file. Of course, this does not include the cost of the database itself. But, given that same database will be used over and over, and typically by many applications, that cost tends to be _amortized_.